# Python II - Assignment 3
This **Home Assignment** is to be submitted and you will be given points for each of the tasks. The first task will recap a lot of the things you learned in this couse (web scraping, regex, pandas, nltk). The second task focuses primarly on graphs.

## Formalities
**Submit in a group of 2-3 people until 06.07.2020 23:59CET. The deadline is strict!**

## Evaluation and Grading
General advice for programming excercises at *CSSH*:
Evaluation of your submission is done semi automatically. Think of it as this notebook being 
executed once. Afterwards, some test functions are appended to this file and executed respectively.

Therefore:
* Submit valid _Python3_ code only!
* We run python 3.8 on our machines, so you can use newer features
* Use external libraries only when specified by task.
* Ensure your definitions (functions, classes, methods, variables) follow the specification if
  given. The concrete signature of e.g. a function usually can be inferred from task description, 
  code skeletons and test cases.
* Ensure the notebook does not rely on current notebook or system state!
  * Use `Kernel --> Restart & Run All` to see if you are using any definitions, variables etc. that 
    are not in scope anymore.
  * Double check if your code relies on presence of files or directories other than those mentioned
    in given tasks. Tests run under Linux, hence don't use Windows style paths 
    (`some\path`, `C:\another\path`). Also, use paths only that are relative to and within your
    working directory (OK: `some/path`, `./some/path`; NOT OK: `/home/alice/python`, 
    `../../python`).
* Keep your code idempotent! Running it or parts of it multiple times must not yield different
  results. Minimize usage of global variables.
* Ensure your code / notebook terminates in reasonable time.

**There's a story behind each of these points! Don't expect us to fix your stuff!**

Regarding the scores, you will get no points for a task if:
- your function throws an unexpected error (e.g. takes the wrong number of arguments)
- gets stuck in an infinite loop
- takes much much longer than expected (e.g. >1s to compute the mean of two numbers)
- does not produce the desired output (e.g. returns an descendingly sorted list even though we asked for ascending, returns the mean and the std even though we asked for the mean only, only prints the output instead of returning it!)
- ...

# Task 1: Sentiment of Tweets (6 points)

## a) Load dataset from the internet (2.0)
Consider the twitter data provided in https://github.com/zfz/twitter_corpus/blob/master/full-corpus.csv.

Write a function `load_dataset()` that gets the dataset from the internet and loads it into a dataframe with two columns `'gt_sentiment'` and `'original_tweet'` that stores the sentiment and (unprocessed) tweet content from that dataset. It returns that dataframe. Use the requests library to fetch the csv file from the internet. Thereby drop all tweets with "irrelevant" sentiment.

In [1]:
import requests
from io import StringIO
import pandas as pd


In [2]:
def load_dataset():
    url="https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv"
    results = requests.get(url).content
    df = pd.read_csv(StringIO(results.decode('utf-8')))
    df=df.loc[:,["Sentiment","TweetText"]]
    df=df.rename(columns={"Sentiment":"gt_sentiment","TweetText":"original_tweet"})
    df=df[df.gt_sentiment!="irrelevant"]
    return df 

In [3]:
df = load_dataset()
df.head()
# example dataframe:
# 	gt_sentiment 	original_tweet
#0 	positive 	Now all @Apple has to do is get swype on the i...
#1 	positive 	@Apple will be adding more carrier support to ...
#2 	positive 	Hilarious @youtube video - guy does a duet wit...
#3 	positive 	@RIM you made it too easy for me to switch to ...
#4 	positive 	I just realized that the reason I got into twi...

,gt_sentiment,original_tweet
0,positive,Now all @Apple has to do is get swype on the i...
1,positive,@Apple will be adding more carrier support to ...
2,positive,Hilarious @youtube video - guy does a duet wit...
3,positive,@RIM you made it too easy for me to switch to ...
4,positive,I just realized that the reason I got into twi...


## b) Clean tweets (1.5 +0.5)
Write a function `clean_tweet(tweet)` that takes a tweet (as string) as input and removes usernames, hashtags and links. Additionally provide a function `clean_all_tweets(df)` that takes a dataframe like above and applies the `clean_tweet` function to all of them. It returns the same dataframe with now an additional column `cleaned_tweet`.

In [4]:
import re

In [5]:
# def clean_tweet(tweet):
#     L=tweet.split()
#     cleaned_tweet=""
#     user_handles=[]
#     hashtags=[]
#     links=[]
#     for k in L:
#         if k.startswith("@"):
#             user_handles.append(k)
#         elif k.startswith("http://"):
#             links.append(k)
#         elif k.startswith("#"):
#             hashtags.append(k)
#         else:
#             cleaned_tweet+=k+" "
      
#     return cleaned_tweet.strip(), user_handles, hashtags, links

In [6]:
def clean_tweet(tweet):

    user_handles = re.findall('@\S+',tweet)
    cleaned_tweet = re.sub('@\S+','',tweet)
    
    hashtags = re.findall('#\S+',tweet)
    cleaned_tweet = re.sub('#\S+','',cleaned_tweet)
    
    # links can be started by "http://" or https:// so we have to remove both of them
    links = re.findall('(?:http://|https://)\S+',tweet)
    cleaned_tweet = re.sub('(?:http://|https://)\S+','',cleaned_tweet)
    
    return cleaned_tweet.strip(), user_handles, hashtags, links

In [7]:
example1="Digital X Worldwide | Today Is Steve Jobs Day In California @apple https://t.co/QSCHuMIN"
example2="Wow. Great deals on refurbed #iPad (first gen) models. RT: Apple offers great deals on refurbished 1st-gen iPads http://t.co/ukWOKBGd @Apple"

In [8]:
print(clean_tweet(example1))
# ('Digital X Worldwide | Today Is Steve Jobs Day In California', ['@apple'], [], ['http://t.co/QSCHuMIN'])
print(clean_tweet(example2))
# ('Wow. Great deals on refurbed  (first gen) models. RT: Apple offers great deals on refurbished 1st-gen iPads', ['@Apple'], ['#iPad'], ['http://t.co/ukWOKBGd'])

('Digital X Worldwide | Today Is Steve Jobs Day In California', ['@apple'], [], ['https://t.co/QSCHuMIN'])
('Wow. Great deals on refurbed  (first gen) models. RT: Apple offers great deals on refurbished 1st-gen iPads', ['@Apple'], ['#iPad'], ['http://t.co/ukWOKBGd'])


In [9]:
def clean_all_tweets(df):
    df["cleaned_tweet"]=df["original_tweet"].apply(lambda x: clean_tweet(x)[0])
    return df

In [10]:
clean_all_tweets(df)

,gt_sentiment,original_tweet,cleaned_tweet
0,positive,Now all @Apple has to do is get swype on the i...,Now all has to do is get swype on the iphone ...
1,positive,@Apple will be adding more carrier support to ...,will be adding more carrier support to the iPh...
2,positive,Hilarious @youtube video - guy does a duet wit...,Hilarious video - guy does a duet with 's Si...
3,positive,@RIM you made it too easy for me to switch to ...,you made it too easy for me to switch to iPho...
4,positive,I just realized that the reason I got into twi...,I just realized that the reason I got into twi...
...,...,...,...
4537,neutral,"@madtruckman 'Modern Day Autograph"", I like th...","'Modern Day Autograph"", I like the way you put..."
4538,neutral,62 Ways to Use #Twitter for Business: http://t...,62 Ways to Use for Business:
4539,neutral,"Log off #Facebook On #Twitter , But I Think i'...","Log off On , But I Think i'm bout to going t..."
4540,neutral,"""#twitter's dumb, I don't like it."" Hush up, ...",""" dumb, I don't like it."" Hush up, Justin."


## c) Predict the sentiment (1.5)
Write a function `calculate_sentiment(df)` that calculate the sentiment of the tweet using the vader sentiment classifier from nltk. Use the **compound** score of the `polarity_scores`. It is between -1 and 1. You can classify the sentiment as - 

```
[-1,-0.2) -> negative

[-0.2,0.2] -> neutral

(0.2,1.0] -> positive```

In [11]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [12]:
def calculate_sentiment(df):
    #function to make things easier
    def sentiment(sent):
        sia = SentimentIntensityAnalyzer()
        d=sia.polarity_scores(sent)
        if -1<=d["compound"]<-0.2:
            return "negative"
        elif -0.2<=d["compound"]<0.2:
            return "neutral"
        else:
            return "positive"
    # use apply
    df["sentiment"]=df["cleaned_tweet"].apply(sentiment)    
    
    return df

In [13]:
calculate_sentiment(df)

,gt_sentiment,original_tweet,cleaned_tweet,sentiment
0,positive,Now all @Apple has to do is get swype on the i...,Now all has to do is get swype on the iphone ...,neutral
1,positive,@Apple will be adding more carrier support to ...,will be adding more carrier support to the iPh...,positive
2,positive,Hilarious @youtube video - guy does a duet wit...,Hilarious video - guy does a duet with 's Si...,positive
3,positive,@RIM you made it too easy for me to switch to ...,you made it too easy for me to switch to iPho...,positive
4,positive,I just realized that the reason I got into twi...,I just realized that the reason I got into twi...,positive
...,...,...,...,...
4537,neutral,"@madtruckman 'Modern Day Autograph"", I like th...","'Modern Day Autograph"", I like the way you put...",positive
4538,neutral,62 Ways to Use #Twitter for Business: http://t...,62 Ways to Use for Business:,neutral
4539,neutral,"Log off #Facebook On #Twitter , But I Think i'...","Log off On , But I Think i'm bout to going t...",neutral
4540,neutral,"""#twitter's dumb, I don't like it."" Hush up, ...",""" dumb, I don't like it."" Hush up, Justin.",negative


## d) Tiny evaluation (0.5)
Write a function `accuracy` that calculates the accuracy of the sentiment classifier by calculating the fraction of cases where the calculated sentiment matches the original sentiment.

In [14]:
def accuracy(df):
    return sum(df["gt_sentiment"]==df["sentiment"])/len(df)

accuracy(df)
# output
# 0.5791471962616822

0.5803154205607477

# Task 2: Minimum cost spanning-tree (7.0 points)

## a) Read an example file (1)
Consider the data provided in file `'reachability.txt'`. It is consists of a network with nodes representing airports of USA and Canada. Edges are weighted so that there is an edge from city i to city j if the estimated airline travel time is less than a threshold. The travel time includes estimated stopover delays.
Write a function `read_graph(filename)` that reads the file at `filename` an returns a directed networkx graph with weights.

In [15]:
import pandas as pd
import networkx as nx

In [16]:
def read_graph(filename):
    L=[]
    with open(filename) as df:
        for line in df:
            L.append(line.strip().split())
    data=L[6:]
    df=pd.DataFrame(data)
    g = nx.DiGraph()
    for i in df.itertuples():
        g.add_edge(int(i[1]), int(i[2]), weight=-int(i[3]))
    return g

In [17]:
G_large = read_graph('reachability.txt')
assert G_large.is_directed()
l = list(G_large.edges(data=True))
l[:5]
# Output
#[(27, 0, {'weight': 757}),
# (27, 1, {'weight': 274}),
# (27, 2, {'weight': 281}),
# (27, 3, {'weight': 205}),
# (27, 4, {'weight': 322})]

[(27, 0, {'weight': 757}),
 (27, 1, {'weight': 291}),
 (27, 2, {'weight': 295}),
 (27, 3, {'weight': 341}),
 (27, 4, {'weight': 359})]

## b) Make the Graph symmetric (1.5)
The graph is directed and the weights are asymmetric. Write a function `make_symmetric(G_in)` that takes such a directed graph with weights as input and converts it to an undirected and symmetric network graph. You should use the following simple strategy:

If for two nodes u,v , edge u->v has a weight $w_1$ and v->u has weight $w_2$ then symmetric network should have one edge u,v with weight $(w_1 + w_2)/2$. Note that in the final graph each edge should be counted only once.

In [18]:
def make_symmetric(G_in):
    g=nx.Graph()
    nodes=list(G_in.nodes())
    edges=list(G_in.edges())
    while len(nodes)!=0:
        node=nodes[0]
        nodes_2=list(G_in.neighbors(node))
        for node_2 in nodes_2:
            if (node_2,node) in edges:
                g.add_edge(node,node_2,weight=(G_in[node][node_2]["weight"]+G_in[node_2][node]["weight"])/2)
            else:
                g.add_edge(node,node_2,weight=G_in[node][node_2]["weight"])
        nodes.remove(node)
    return g
                
            
        


In [19]:

# Create example Graph
G1 = nx.DiGraph()
G1.add_edge(0,1,weight=10)
G1.add_edge(1,0,weight=5)
G1.add_edge(0,2,weight=3)
G1.add_edge(3,0,weight=1)
G1.add_edge(2,3,weight=19)
G1.add_edge(3,2,weight=11)
G1.add_edge(3,4,weight=0.1)
G_out = make_symmetric(G1)
print(G_out.edges(data=True))
# [(0, 1, {'weight': 7.5}), (0, 2, {'weight': 3}), (0, 3, {'weight': 1}), (2, 3, {'weight': 15.0}), (3, 4, {'weight': 0.1})]

[(0, 1, {'weight': 7.5}), (0, 2, {'weight': 3}), (0, 3, {'weight': 1}), (2, 3, {'weight': 15.0}), (3, 4, {'weight': 0.1})]


## c) Helper class: ConnectedComponentsTracker (2.5)
It is sometimes useful to keep track of the connected components https://en.wikipedia.org/wiki/Component_(graph_theory) when building a graph. Therefor write a class `ConnectedComponentsTracker` that keeps track of connected components in its attribute set. This attribute is a list of sets, where if two nodes u,v are in the same sets they are connected, otherwise they are not.
It should have these methods:
1. The constructor should initialize the `self.sets` to an empty list
2. `find_set(self, u)` that returns the set that contains u or `len(self.sets)` if u is not in any of the sets
3. `add_edge(self, u, v)` that updates the sets of the connected component to account for the new edge `(u,v)`
4. `forms_loop(self, u, v)` that returns True if inserting the undirected edge `(u,v)` would create a loop or is an existing edge, False otherwise

The ConnectedComponentsTracker is only intended for use with symmetric Graphs so any edge u,v is can be considered symmetric. You can find example output for a valid `ConnectedComponentsTracker` below.

In [20]:
class ConnectedComponentsTracker:
    def __init__(self):
        self.sets=[]
        
    def find_set(self,u):
        for set_u in self.sets:
            if u in set_u:
                return set_u
        return len(self.sets)
    
    def add_edge(self,u,v):
        k=True
        n=0
        for s in self.sets:
            if u in s :
                n+=1
                if n==2:
                    set_=s
                else:
                    s.add(v)
                    k=False
                
            elif v in s:
                n+=1
                if n==2:
                    set_=s
                else:
                    s.add(u)
                    k=False
        if k :
            self.sets.append({u,v})
        if n==2:
            self.sets.remove(set_)
            for s in self.sets:
                if u in s:
                    s.update(set_)
            
        
    
    def forms_loop(self,u,v):
        for s in self.sets:
            if u in s and v in s:
                return True
        return False

            


In [21]:
# Test your class here
c=ConnectedComponentsTracker()

c.add_edge(0,1)
print(0, c.sets)
print(0.1, c.forms_loop(2,1))
c.add_edge(1,2)
print(1, c.sets)

c.add_edge(3,2)
print(2, c.sets)
print(2.1, c.forms_loop(1,3))

c.add_edge(1,3)
print(3, c.sets)

c.add_edge(4,5)
print(4, c.sets)

c.add_edge(5,6)
print(5, c.sets)
print(5.1, c.forms_loop(4,6))
print(5.2, c.forms_loop(1,6))
print(5.3, c.forms_loop(1,10))

c.add_edge(2,6)
print(6, c.sets)

c.add_edge(9,10)
print(7, c.sets)

c.add_edge(10,3)
print(8, c.sets)

# --- Output ---
# Note that the order in a set is not deterministic!
# 0   [{0, 1}]
# 0.1 False
# 1   [{0, 1, 2}]
# 2   [{0, 1, 2, 3}]
# 2.1 True
# 3   [{0, 1, 2, 3}]
# 4   [{0, 1, 2, 3}, {4, 5}]
# 5   [{0, 1, 2, 3}, {4, 5, 6}]
# 5.1 True
# 5.2 False
# 5.3 False
# 6   [{0, 1, 2, 3, 4, 5, 6}]
# 7   [{0, 1, 2, 3, 4, 5, 6}, {9, 10}]
# 8   [{0, 1, 2, 3, 4, 5, 6, 9, 10}]


0 [{0, 1}]
0.1 False
1 [{0, 1, 2}]
2 [{0, 1, 2, 3}]
2.1 True
3 [{0, 1, 2, 3}]
4 [{0, 1, 2, 3}, {4, 5}]
5 [{0, 1, 2, 3}, {4, 5, 6}]
5.1 True
5.2 False
5.3 False
6 [{0, 1, 2, 3, 4, 5, 6}]
7 [{0, 1, 2, 3, 4, 5, 6}, {9, 10}]
8 [{0, 1, 2, 3, 4, 5, 6, 9, 10}]


## d) Kruskal's algorithm (1.5)
Obtain minimum cost spanning tree (https://en.wikipedia.org/wiki/Minimum_spanning_tree) using Kruskal's algorithm. The algorithm is given below - 

1. Sort all the edges in non-decreasing order of their weight.

2. Pick the smallest edge (i.e., the edge with the lowest weight). Check if it forms a cycle with the spanning tree formed so far (i.e., all edges selected so far). If cycle is not formed, include this edge. Else, discard it.

3. Repeat step#2 until there are (V-1) edges in the spanning tree. 

V -> number of nodes in the graph

Now write a function `minSpanTree` that takes an undirected networkx Graph with weights and returns a new undirected networkx Graph with only those edges that you obtained from Kruskal's algorithm.

In [22]:
list(G_out.edges(data=True))

[(0, 1, {'weight': 7.5}),
 (0, 2, {'weight': 3}),
 (0, 3, {'weight': 1}),
 (2, 3, {'weight': 15.0}),
 (3, 4, {'weight': 0.1})]

In [24]:
def minSpanTree(G_in):
    g=nx.Graph()
    edges=list(sorted(G_in.edges(data=True), key=lambda t: t[2].get('weight')))
    n=len(list(G_in.nodes()))
    g.add_edge(edges[0][0],edges[0][1],weight=edges[0][2]["weight"])
    k=1
    for i in range(1,n):
        edge=edges[i]
        tree=list(g.nodes())
        if edge[0] in tree and edge[1] in tree:
            continue
        elif edge[0] in tree:
            g.add_edge(edges[i][0],edges[i][1],weight=edges[i][2]["weight"])
            k+=1
        elif edge[1] in tree:
            g.add_edge(edges[i][0],edges[i][1],weight=edges[i][2]["weight"])
            k+=1
        else:
            if k==n-1:
                break
    return g
        

G = minSpanTree(G_out)

list(G.edges(data=True))
# [(3, 4, {'weight': 0.1}), (3, 0, {'weight': 1}), (0, 2, {'weight': 3}), (0, 1, {'weight': 7.5})]

[(3, 4, {'weight': 0.1}),
 (3, 0, {'weight': 1}),
 (0, 2, {'weight': 3}),
 (0, 1, {'weight': 7.5})]

Use the `minimum_spanning_tree` function of networkx to obtain the minimum cost spanning tree for the graph.

In [25]:
def minSpanTreeNetworkx(G):
    return nx.minimum_spanning_tree(G)
G_netx = minSpanTreeNetworkx(G_out)

In [26]:
list(G_netx.edges(data=True))

[(0, 3, {'weight': 1}),
 (0, 2, {'weight': 3}),
 (0, 1, {'weight': 7.5}),
 (3, 4, {'weight': 0.1})]

## e) Compare the results (0.25+0.25)
Compare your minimal spanning tree with the one from networkx.
Therefor write a function `compare_edges(G, G_netx)` that returns the extra and missing edges for the two graphs.
Additionally write a function `get_total_span(G)` that returns the total span (sum of all weights) for a Graph.
You can use these functions to verify your implementation. Make sure that if there are missing or extra edges that the total span of your implementation is the same as that of networkx.

In [27]:
def get_total_span(G):
    s=0
    for t in list(G.edges(data=True)):
        s+=t[2]["weight"]           
    return s
    

def compare(G, G_netx):
    extra_edges = []
    missing_edges = []
    edges_G=list(G.edges())
    edges_G_netx=list(G_netx.edges())
    for edge in edges_G:
        if edge not in edges_G_netx:
            extra_edges.append(edge)
    for edge in edges_G_netx:
        if edge not in edges_G:
            missing_edges.append(edge)
    
    return extra_edges, missing_edges

In [28]:
get_total_span(G_netx)

11.6

In [29]:
G_sym = make_symmetric(G_large)
G = minSpanTree(G_sym)
G_netx = minSpanTreeNetworkx(G_sym)
compare(G, G_netx) # There might be some confused edges

([(88, 428),
  (428, 383),
  (428, 207),
  (428, 203),
  (383, 316),
  (394, 45),
  (394, 331),
  (207, 429),
  (207, 57),
  (45, 182)],
 [(27, 298),
  (27, 323),
  (0, 57),
  (1, 437),
  (1, 195),
  (2, 94),
  (3, 76),
  (4, 100),
  (5, 19),
  (6, 294),
  (7, 126),
  (7, 115),
  (7, 13),
  (7, 248),
  (7, 419),
  (8, 178),
  (9, 445),
  (9, 323),
  (10, 64),
  (10, 100),
  (11, 379),
  (11, 102),
  (13, 79),
  (14, 208),
  (14, 426),
  (14, 89),
  (14, 124),
  (14, 218),
  (14, 37),
  (14, 105),
  (14, 220),
  (14, 199),
  (15, 268),
  (16, 68),
  (17, 100),
  (18, 68),
  (19, 250),
  (19, 85),
  (19, 71),
  (19, 40),
  (19, 180),
  (19, 217),
  (19, 279),
  (19, 107),
  (19, 21),
  (19, 381),
  (19, 407),
  (19, 51),
  (19, 427),
  (19, 87),
  (19, 142),
  (19, 314),
  (19, 265),
  (19, 272),
  (19, 319),
  (19, 213),
  (19, 289),
  (22, 357),
  (22, 46),
  (23, 178),
  (23, 94),
  (23, 267),
  (26, 246),
  (28, 46),
  (29, 357),
  (32, 178),
  (33, 178),
  (35, 383),
  (36, 412),
  

In [31]:
# If there are confused edges the total span should be the same still
print(get_total_span(G)) 
print(get_total_span(G_netx))
# output
# 23864.0

720.5
23864.0
